# Project - Airline AI Assistant

We'll now bring together what we've learned to make an AI Customer Support assistant for an Airline

In [2]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [3]:
# Initialization

load_dotenv()

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
MODEL = "gpt-4o-mini"
openai = OpenAI()

# As an alternative, if you'd like to use Ollama instead of OpenAI
# Check that Ollama is running for you locally (see week1/day2 exercise) then uncomment these next 2 lines
# MODEL = "llama3.2"
# openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')


OpenAI API Key exists and begins sk-proj-


In [4]:
system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."

In [5]:
# This function looks rather simpler than the one from my video, because we're taking advantage of the latest Gradio updates

def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages)
    return response.choices[0].message.content

gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7887

To create a public link, set `share=True` in `launch()`.


## Tools

Tools are an incredibly powerful feature provided by the frontier LLMs.

With tools, you can write a function, and have the LLM call that function as part of its response.

Sounds almost spooky.. we're giving it the power to run code on our machine?

Well, kinda.

In [6]:
# Let's start by making a useful function

ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")

In [7]:
get_ticket_price("Berlin")

Tool get_ticket_price called for Berlin


'$499'

In [8]:
# There's a particular dictionary structure that's required to describe our function:

price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [9]:
# And this is included in a list of tools:

tools = [{"type": "function", "function": price_function}]

## Getting OpenAI to use our Tool

There's some fiddly stuff to allow OpenAI "to call our tool"

What we actually do is give the LLM the opportunity to inform us that it wants us to run the tool.

Here's how the new chat function looks:

In [15]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    # ai runs but stops with 'tool_calls' as finish reason when it is asking for the tool to be called before it can make it's answer to the user.
    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message # the message and context leading to the tool being requested
        response, city = handle_tool_call(message) # function (below) able to handle the prev defined tool being called
        # print((message, response))
        messages.append(message) # adding the needed info to the messages ready to rerun ai now with all the tool call info requested
        messages.append(response) # adding the tool call info requested
        response = openai.chat.completions.create(model=MODEL, messages=messages) #rerun
    
    return response.choices[0].message.content

In [16]:
# We have to write that function handle_tool_call:

def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    city = arguments.get('destination_city')
    price = get_ticket_price(city)
    response = {
        "role": "tool",
        "content": json.dumps({"destination_city": city,"price": price}),
        "tool_call_id": tool_call.id
    }
    return response, city

In [17]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7889

To create a public link, set `share=True` in `launch()`.


Tool get_ticket_price called for Berlin
(ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_fq4njq6QEZElql5vlnNoAfNw', function=Function(arguments='{"destination_city":"Berlin"}', name='get_ticket_price'), type='function')]), {'role': 'tool', 'content': '{"destination_city": "Berlin", "price": "$499"}', 'tool_call_id': 'call_fq4njq6QEZElql5vlnNoAfNw'})


In [24]:
def counter(item_to_count, phrase_count_within):
    '''
    counts the number of instances of the supplied item in the supplied phrase 
    '''
    if item_to_count == 'words':
        # count the number of words
        return len(phrase_count_within.split(' '))
    else:
        # count specific phrase/word/char/s
        return phrase_count_within.count(item_to_count)

In [22]:
counter('r','strawberry')

3

In [25]:
counter('words','strawberry')

1

In [26]:
counter('words','many many words')

3

In [45]:
# There's a particular dictionary structure that's required to describe our function:

counter_function = {
    "name": "counter",
    "description": "Get the count of the number of instances of the supplied item_to_count text within the supplied phrase_count_within . Call this whenever you need to know the number of letters within a string, \
    for example when a customer asks 'How many 'r' are in strawberry' or 'how many words are in this sentence' for the latter, you will supply 'words' as the item_to_count, this is a special argument that forces \
    the number of words to be counted instead of the instances of 'words' literrally!",
    "parameters": {
        "type": "object",
        "properties": {
            "item_to_count": {
                "type": "string",
                "description": "The text that needs to be searched for",
            },
            "phrase_count_within": {
                "type": "string",
                "description": "The text that is searched within",
            },
        },
        "required": ["item_to_count", "phrase_count_within"],
        "additionalProperties": False
    }
}

In [46]:
# And this is included in a list of tools:

tools2 = [{"type": "function", "function": counter_function}]
system_message2 = "you are an assistant who answers in show courteous one liners."

In [47]:
def chat2(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools2)

    # ai runs but stops with 'tool_calls' as finish reason when it is asking for the tool to be called before it can make it's answer to the user.
    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message # the message and context leading to the tool being requested
        response, item_to_count, phrase_count_within = handle_tool_call2(message) # function (below) able to handle the prev defined tool being called
        # print((message, response))
        messages.append(message) # adding the needed info to the messages ready to rerun ai now with all the tool call info requested
        messages.append(response) # adding the tool call info requested
        response = openai.chat.completions.create(model=MODEL, messages=messages) #rerun
    
    return response.choices[0].message.content

In [48]:
# We have to write that function handle_tool_call:

def handle_tool_call2(message):
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    item_to_count = arguments.get('item_to_count')
    phrase_count_within = arguments.get('phrase_count_within')
    count = counter(item_to_count, phrase_count_within)
    response = {
        "role": "tool",
        "content": json.dumps({"item_to_count": item_to_count, "phrase_count_within": phrase_count_within, "count": count}),
        "tool_call_id": tool_call.id
    }
    print(response)
    return response, item_to_count, phrase_count_within

In [49]:
gr.ChatInterface(fn=chat2, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7895

To create a public link, set `share=True` in `launch()`.


{'role': 'tool', 'content': '{"item_to_count": "words", "phrase_count_within": "how many words are in this sentence?", "count": 7}', 'tool_call_id': 'call_WirVR7KBPHXaW5BvLQgtL0v6'}
